# Einführung in Machine Learning mit Python

## Vektorisierung

### Datensets mit Features - Beispiel Iris-Datenset

Einige Datensets sind bereits schon in `scikit-learn` eingebaut. Du brauchst dich nicht mehr darum zu kümmern, diese
* herunterzuladen,
* zu verifizieren und
* zu vektorisieren

Das [Iris-Datenset](https://en.wikipedia.org/wiki/Iris_flower_data_set) ist ein *Klassiker*:

![image](https://upload.wikimedia.org/wikipedia/commons/thumb/5/56/Kosaciec_szczecinkowaty_Iris_setosa.jpg/220px-Kosaciec_szczecinkowaty_Iris_setosa.jpg) ![image](https://upload.wikimedia.org/wikipedia/commons/thumb/4/41/Iris_versicolor_3.jpg/220px-Iris_versicolor_3.jpg) ![image](https://upload.wikimedia.org/wikipedia/commons/thumb/9/9f/Iris_virginica.jpg/220px-Iris_virginica.jpg)

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

Leider in einem etwas sperrigen Format

In [ ]:
iris

Besser als `DataFrame` mit `pandas`

In [ ]:
import pandas as pd
idf = pd.DataFrame(iris["data"], columns=["Sepal Length", "Sepal Width", "Petal Length", "Petal Width"])
idf["target"] = iris["target"]
idf["name"] = [iris["target_names"][target] for target in iris["target"]]
idf

*Five Number Summaries*

In [ ]:
idf.describe()

Verteilungen als Histogramm plotten

In [ ]:
idf["Petal Width"].plot.hist(bins=20)

Überlegung, welche Features sich besonders zur Differenzierung eignen:

In [ ]:
idf.plot.scatter(x="Sepal Length", y="Sepal Width", c="target", cmap="Set1")

Oder alle auf einmal? Hier erkennst du mögliche Korrelationen oder in welchen Dimensionen die Objekte sich am deutlichsten unterscheiden:

In [ ]:
import seaborn as sns
sns.pairplot(idf.drop(columns=["target"]), hue="name")

### Unstrukturierte Datensets - Feature Engineering

Bei unstrukturierten Daten musst du erst Features finden. Bei Bildern ist das z.B. sehr schwierig, deswegen wird dort fast immer nur Deep Learning verwendet. Dort macht der erste Layer aus den Pixel eine Art Features.

Wir beschäftigen und jetzt mit Textdaten. Dazu haben wir alle Toplevel-Posts des [Technology Subreddit](https://www.reddit.com/r/Technology) heruntergeladen und in einer CSV-Datei gespeichert:

In [ ]:
docs = pd.read_csv("https://github.com/datanizing/heise-webinar-ml-python/raw/master/reddit-technology-toplevel-title.csv.gz", 
                   parse_dates=["created_utc"])

Wie du siehst, sind das *viele* Posts, nämlich fast zwei Millionen:

In [ ]:
docs

In [ ]:
docs.dtypes

Sind die Daten hinreichend aktuell? Solche Informationen überprüfst du besser gleich ganz zu Beginn:

In [ ]:
docs.set_index("created_utc").resample("Q").count().plot()

Das ist zwar nicht perfekt, aber es eignet sich doch hinreichend gut für eine Analyse.

Um die Daten zu vektorisieren, solltest du zuerst die Stopwords eliminieren. Das ist normalerweise ein iterativer Prozess, hier findest du die etwas ergänzte Stopword-Liste von `spacy` (damit du das nicht noch installieren musst):

In [ ]:
stop_words= {"'d", "'ll", "'m", "'re", "'s", "'ve", 'a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 
             'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 
             'amongst', 'amount', 'amp', 'an', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 
             'anywhere', 'are', 'around', 'as', 'at', 'back', 'be', 'became', 'because', 'become', 'becomes', 
             'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 
             'beyond', 'blog', 'body', 'both', 'bottom', 'but', 'buy', 'buycheap', 'by', 'ca', 'call', 'can', 'cannot', 
             'case', 'change', 'co', 'com', 'could', 'create', 'delete', 'did', 'do', 'does', 'doing', 'done', 'down', 
             'download', 'drive', 'due', 'during', 'each', 'eight', 'either', 'eleven', 'else', 'elsewhere', 'email', 
             'empty', 'enough', 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 'except', 'few', 
             'fifteen', 'fifty', 'first', 'five', 'fix', 'for', 'former', 'formerly', 'forty', 'four', 'from', 'front', 
             'full', 'further', 'get', 'give', 'go', 'good', 'had', 'has', 'have', 'he', 'help', 'hence', 'her', 'here', 
             'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'however', 
             'http', 'https', 'hundred', 'i', 'if', 'in', 'indeed', 'into', 'is', 'it', 'its', 'itself', 'just', 'keep', 
             'last', 'late', 'latter', 'latterly', 'least', 'less', 'll', 'look', 'made', 'make', 'many', 'market', 'may', 
             'me', 'meanwhile', 'message', 'might', 'mine', 'more', 'moreover', 'most', 'mostly', 'move', 'much', 
             'must', 'my', 'myself', "n't", 'name', 'namely', 'need', 'neither', 'never', 'nevertheless', 'new', 'news', 
             'next', 'nine', 'no', 'nobody', 'none', 'noone', 'nor', 'not', 'nothing', 'now', 'nowhere', 'number', 'n‘t', 
             'n’t', 'of', 'off', 'often', 'on', 'once', 'one', 'online', 'only', 'onto', 'or', 'other', 'others', 
             'otherwise', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 'page', 'part', 'pass', 'per', 'perhaps', 
             'please', 'post', 'put', 'question', 'quite', 'rather', 're', 'really', 'reddit', 'regarding', 'remove', 
             'review', 'same', 'say', 'search', 'see', 'seem', 'seemed', 'seeming', 'seems', 'self', 'send', 'serious', 
             'several', 'she', 'should', 'show', 'side', 'since', 'site', 'six', 'sixty', 'so', 'some', 'somehow', 
             'someone', 'something', 'sometime', 'sometimes', 'somewhere', 'still', 'such', 'support', 'take', 'ten', 
             'test', 'text', 'than', 'that', 'the', 'their', 'them', 'themselves', 'then', 'thence', 'there', 
             'thereafter', 'thereby', 'therefore', 'therein', 'thereupon', 'these', 'they', 'third', 'this', 'those', 
             'though', 'three', 'through', 'throughout', 'thru', 'thus', 'time', 'to', 'together', 'too', 'top', 
             'toward', 'towards', 'twelve', 'twenty', 'two', 'under', 'unless', 'unlock', 'until', 'up', 'upon', 
             'us', 'use', 'used', 'using', 'various', 've', 'very', 'via', 'video', 'was', 'watch', 'way', 'we', 'well', 
             'were', 'what', 'whatever', 'when', 'whence', 'whenever', 'where', 'whereafter', 'whereas', 'whereby', 
             'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', 'whoever', 'whole', 
             'whom', 'whose', 'why', 'will', 'with', 'within', 'without', 'work', 'would', 'yet', 'you', 'your', 
             'yours', 'yourself', 'yourselves', '‘d', '‘ll', '‘m', '‘re', '‘s', '‘ve', '’d', '’ll', '’m', '’re', 
             '’s', '’ve'}

Für die Vektorisierung von Dokumente werden normalerweise die Wörter gezählt und als Features verwendet. Damit besonders häufige Wörter nicht zu stark dominieren, werden die mit der sog. "Inverierten Dokumentenfrequenz" abgewertet.

`scikit-learn` kann das mit dem `TfidfVectorizer` alles für dich erledigen

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(min_df=10, stop_words=stop_words)
tfidf_vectors = tfidf_vectorizer.fit_transform(docs["title"].map(str))
tfidf_vectors

Wie du siehst, ist die entstehende Matrix mit fast zwei Millionen Zeilen und knap 45.000 Spalten *sehr groß*. Da die meisten Wörter in den meisten Dokumenten nicht vorkommen, besteht sie zu einem ganz großen Teil aus leeren Einträgen und kann daher als sog. *sparse matrix* sehr effizient abgelegt werden:

In [ ]:
tfidf_vectors.data.nbytes

Bei einer `float`-Darstellung mit vier Bytes pro `float` würde das in einer naiven Darstellung 1922041 x 44980 x 4 Bytes = 322 GB RAM benötigen. Zum Glück sind es hier nur 130 MB - ein Effizienzgewinn fast um den Faktor 3.000. Nur deswegen funktioniert das überhaupt!

## Überwachte Lernverfahren

Überwachte Lernverfahren benötigen Trainingsdaten.

### Iris-Klassifikation

Zum Training nutzt du die Iris-Daten, von denen du das Ergebnis schon kennst. Als Modell verwendest du eine sog. [Support Vector Machine](https://de.wikipedia.org/wiki/Support_Vector_Machine). Diese funktioniert sehr effizient:

In [ ]:
from sklearn.linear_model import SGDClassifier
svm = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, random_state=42)
svm.fit(iris['data'], iris['target'])

Jetzt kannst du aus den Daten vorhersagen lassen, zu welcher Iris-Sorte eine Pflanze gehört, wenn du nur die Messwerte kennst.

In [ ]:
svm.predict(iris['data'])

Wie vergleicht sich das mit den echten Werten?

In [ ]:
svm.predict(iris['data']) == iris['target']

Eigentlich sieht das ganz gut aus, aber es gibt mehrere Probleme:
* Kann man die Performance des Klassifikators *messen*?
* Hat der Klassifikator evtl. nur (fast) alles auswendig gelernt oder kann er tatsächlich *abstrahieren*?

### Trainings-Test-Split

Um das zu beheben, kannst du die vorklassifizierte Datenmenge in zwei Teile zerlegen: eine *Trainingsmenge*, die nur dem Training des Klassifikators dient und eine davon unabhängige *Testmenge*, mit der der Klassifikator beweisen kann, wie gut er abstrahieren kann.

`scikit-learn` stellt dafür eine Funktion `train_test_split` zur Verfügung. Der `random_state` dient der Reproduzierbarkeit, mit `test_size` kann man das Verhältnis der beiden Mengengrößen wählen.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris['data'], iris['target'], test_size = 0.25, random_state = 42)

Wieder trainierst du die SVM als Klassifikator:

In [ ]:
from sklearn.svm import SVC
svm = SVC(max_iter=1000, tol=1e-3, random_state=42)
svm.fit(X_train, y_train)

Dieses Mal sagst du die Spezies für die Testdaten vorher:

In [ ]:
svm.predict(X_test) == y_test

Zwei Werte sind falsch vorhergesagt. Welche das sind, findest du in der [Confusion Matrix](https://en.wikipedia.org/wiki/Confusion_matrix):

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, svm.predict(X_test))

Auf der Diagonalen stehen dort die richtig vorgergesagten Werte, die `2` außerhalb der Diagonale zeigt dir falsch vorhergesagte Ergebnisse. In diesem Fall wäre das richtige Ergebnis die Klasse `2` gewesen (2. Spalte), stattdessen wurde `3` vorhergesagt (3. Zeile).

Oftmals möchtest du die Genauigkeit der Klassifikation messen. Dazu dienen gleich zwei Größen, nämlich die Precision (Spezifizität) und der Recall (Sensitivität):

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, svm.predict(X_test)))

In [ ]:
print(classification_report(y_train, svm.predict(X_train)))

Im Gegensatz zur häufig verwendeten *Accuracy* können Precision und Recall auch mit *schiefen* Verteilungen umgehen.

### Reddit-Klassifikation

Betrachten wir unser echtes Datenset, gibt es auch hier Kategorien, nämlich die Flairs:

In [ ]:
docs.groupby("flair").agg({"title": "count"}).sort_values("title", ascending=False).head(20)

Passen zum Thema des Webinars suchen wir uns den Flair *Artificial Intelligence* aus und setzen dessen *Target* auf 1:

In [ ]:
docs["target"] = 0
docs.loc[docs["flair"] == "Artificial Intelligence", "target"] = 1

Nun sind allerdings die Dokumente ohne diesen Flair deutlich überrepräsentiert:

In [ ]:
ai = docs[docs["flair"] == "Artificial Intelligence"]
len(ai)

In [ ]:
non_ai = docs[(docs["flair"] != "Artificial Intelligence") & 
              (docs["created_utc"].dt.year >= 2015) &
              ~docs["flair"].isna()]
len(non_ai)

Damit kann ein Modell nicht gut trainiert werden, weil viel zu viele negative Beispiele den Klassifikator in eine falsche Richtung drängen. Deswegen *stratifizieren* wir das Datenset und nehmen so viele negative wie positive Samples:

In [ ]:
data = pd.concat([ai, non_ai.sample(n = len(ai), random_state=42)])
data

Du kannst weiter mit dem bereits trainierten Vectorizer arbeiten und rufst daher nur dessen `transform`-Methode auf:

In [ ]:
ai_tfidf_vectors = tfidf_vectorizer.transform(data["title"].map(str))

Um die Ergebnisqualität messen zu können, teilst du das Datenset auf:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(ai_tfidf_vectors, data['target'], test_size = 0.25, random_state = 42)

Der Klassifikator ist schnell trainiert:

In [ ]:
ai_svm = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, random_state=42)
ai_svm.fit(X_train, y_train)

Das Ergebnis ist schon ziemlich gut:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, ai_svm.predict(X_test)))

Mit 88% Genauigkeit (hier passt die Accuracy wegen dem gleichverteilten Datenset) kannst du jetzt vorhersagen, ob ein Post von AI handelt oder nicht.

### Reddit-Regression (Vorhersage)

Jetzt willst du versuchen, den Trend in den AI-Meldungen zu ermitteln. Dazu aggregierst du zunächst die AI-Post und parallel dazu alle Posts.

In [ ]:
time_agg = docs.set_index("created_utc").resample("M").\
       agg({"target": "sum", "title": "count" }).rename(columns={"title": "total"})
time_agg

Jetzt kannst du den relativen Anteil der AI-Posts ausrechnen. Nur diese lassen sich vorhersagen, weil sonst das Post-Volumen von Reddit noch mit vorhergesagt werden müsste:

In [ ]:
time_agg["rel"] = time_agg["target"] / time_agg["total"]
time_agg[["rel"]].plot()

Für eine Trendvorhersage erzeugst du einen neuen `DataFrame`:

In [ ]:
pa = pd.DataFrame()
pa["ds"] = time_agg.index
pa["y"] = time_agg["rel"].values
pa

Flairs wurden erst ab 2015 vergeben, daher nutzt du nur diesen Zeitraum:

In [ ]:
pa = pa[pa["ds"] >= '2015-01-01']

Die einfachste Vorhersage ist eine lineare Regression.

In [ ]:
from scipy.stats import linregress
linregress(x=pa["ds"].index.values, y=pa["y"])

Das [Pearson R und P](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) sagen dir, dass hier eine relativ klare Korrelation vorliegt.

Noch viel besser geht das allerdings mit [Prophet](https://facebook.github.io/prophet/):

In [ ]:
!pip install fbprophet
from fbprophet import Prophet

Zuerst instanziierst du ein `Prophet` Objekt und übergibst ihm die Daten:

In [ ]:
m = Prophet()
m.fit(pa)

Dann kümmerst du dich um den Future-`DataFrame`:

In [ ]:
future = m.make_future_dataframe(periods=20, freq='M')

Und führst die Vorhersagen durch:

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

Besonders schön sind die dadurch entstehenden Plots:

In [ ]:
fig1 = m.plot(forecast)

Auch die Analyse von Trend und Saisonalität ist aufschlussreich:

In [ ]:
fig2 = m.plot_components(forecast)

Wir sind zwar im richtigen Jahr unterwegs, aber leider gerade in einem saisonalen Downturn. Das ist allerdings mit Vorsicht zu genießen - bei AI gibt es keine Saisonalität (trotz [AI winter](https://en.wikipedia.org/wiki/AI_winter)).

## Unüberwachte Lernverfahren

Unüberwachte Lernverfahren ermitteln die *intrinsische Struktur* von Daten. Sie benötigen keine Trainingsdaten und sind unvoreingenommen.

### Iris-Clustering

Du kannst die Iris-Daten auch clustern. Die meisten Cluster-Algorithmen brauchen als Vorgabe die Anzahl der Cluster, so auch das hier verwendete `KMeans`. Versuche es erst mit einer "falschen" Anzahl an Clustern:

In [ ]:
from sklearn.cluster import KMeans
km5 = KMeans(n_clusters=5)
km5.fit(iris['data'])

In dem `labels_`-Feld stehen dir jetzt die entdecken Cluster zur Verfügung:

In [ ]:
km5.labels_

Schau dir zum Vergleich die echten Varianten an:

In [ ]:
iris['target']

Die Zahlen sind beliebig. Am Anfang hat es noch ganz gut geklappt - aber dann gibt es schon ein ziemliches Durcheinander!

Oben hattet du gesehen, dass sich die Iris-Varianten am besten durch die *Petal Length* vs. *Sepal Length* unterscheiden lassen. Mit `seaborn` kannst du einen Scatter-Plot zeichnen lassen und sowohl die Marker als auch die Farben beeinflussen. Wenn du die Marker auf die echte Spezies setzt und die Farben auf die Cluster, siehst du, was nicht richtig gelöst wurde:

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16,9))
sns.scatterplot(x=idf["Sepal Length"], y=idf["Petal Length"], hue=idf["name"], style=km5.labels_, 
                s=300)

Versuch es nochmal mit der richtigen Anzahl der Cluster:

In [ ]:
km3 = KMeans(n_clusters=3)
km3.fit(iris['data'])
km3.labels_

Das Ergebnis sieht jetzt auch optisch sehr viel besser aus:

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16,9))
sns.scatterplot(x=idf["Sepal Length"], y=idf["Petal Length"], hue=idf["name"], style=km3.labels_, 
                s=300)

Ein Clusterverfahren wie `MeanShift` ermittelt die Anzahl der Cluster selbstständig:

In [ ]:
from sklearn.cluster import MeanShift
ms = MeanShift()
ms.fit(iris['data'])

In [ ]:
plt.figure(figsize=(16,9))
sns.scatterplot(x=idf["Sepal Length"], y=idf["Petal Length"], hue=idf["name"], style=ms.labels_, 
                s=300)

Mithilfe des Silhouette-Scores kannst du ermitteln, wie gut das Clustering (für *konvexe Cluster*) funktioniert hat:

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_score(iris['data'], km3.labels_)

In [ ]:
silhouette_score(iris['data'], km5.labels_)

In [ ]:
silhouette_score(iris['data'], ms.labels_)

In [ ]:
km2 = KMeans(n_clusters=2)
km2.fit(iris['data'])
km2.labels_

In [ ]:
silhouette_score(iris['data'], km2.labels_)

Innerhalb des gleichen Cluster-Verfahrens kannst du den Silhouette-Score gut vergleichen. `MeanShift` hat nach dem Bild oben natürlich auch gute Argument, warum es nur zwei Cluster gefunden hat. Vermutliche sind sich Spezies in dem "oberen" Cluster auch deutlich ähnlicher!

### Reddit-Topic Models

Topic-Modelle kann `scikit-learn` mit mehreren Methoden berechnen. `NMF` ist deutlich schneller als `LDA` und hat häufig ebenso gute Ergebnisse.

Die Aufrufe sind sehr ähnlich zum Clustering und zur Klassifikation:

In [ ]:
from sklearn.decomposition import NMF

num_topics = 10

nmf = NMF(n_components = num_topics)
nmf.fit(tfidf_vectors)

Um die Ergebnisse darzustellen, wollen wir Wordclouds benutzen. Dazu berechnest du die wichtigsten Wörter in jedem einzelnen Topic:

In [ ]:
!pip install wordcloud
import matplotlib.pyplot as plt
from wordcloud import WordCloud

def wordcloud_topic_model_summary(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        freq = { feature_names[i]: topic[i] for i in topic.argsort()[:-no_top_words - 1:-1] }
        wc = WordCloud(background_color="white", max_words=100, width=960, height=540)
        wc.generate_from_frequencies(freq)
        plt.figure(figsize=(12,12))
        plt.imshow(wc, interpolation='bilinear')

In [ ]:
wordcloud_topic_model_summary(nmf, tfidf_vectorizer.get_feature_names(), 40)

Das Ergebnis ist gut interpretierbar und vor allem *unvoreingenommen*. Mit wenigen Zeilen Code hast du so die Themen in zwei Millionen Posts bestimmt!

Die Anzahl der Topics kannst du variieren und sehen, ob du damit bessere Ergebnisse erzielen kannst. Es gibt auch Scores, mit denen du die Güte ermitteln kannst (Coherence Score oder Perplexity).

## Zeitevolution

In [ ]:
docs["month"] = pd.to_datetime(docs["created_utc"], utc=True).dt.strftime("%Y-%m")

In [ ]:
import numpy as np
month_data = []
for month in np.unique(np.unique(docs["month"])):
    W_month = nmf.transform(tfidf_vectors[np.array(docs["month"] == month)])
    month_data.append([month] + list(W_month.sum(axis=0)/W_month.sum()*100.0))

In [ ]:
topic_names = []
voc = tfidf_vectorizer.get_feature_names()
for topic in nmf.components_:
    important = topic.argsort()
    top_word = voc[important[-1]] + " " + voc[important[-2]]
    topic_names.append("Topic " + top_word)

In [ ]:
df_month = pd.DataFrame(month_data, columns=["month"] + topic_names).set_index("month")
df_month.plot.area(figsize=(16,9))